In [9]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b750da64-a0fa-50a9-eebd-8fa0b15fe2f9)


In [10]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git

fatal: destination path 'pubmed-rct' already exists and is not an empty directory.


In [11]:
!ls pubmed-rct

PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [12]:
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/

dev.txt  test.txt  train.txt


In [13]:
data_dir = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/'

In [14]:
import os

paths_to_files = [data_dir + file_name for file_name in os.listdir(data_dir)]

paths_to_files

['/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

In [15]:
def getlines(filename) -> list:
  """
  Reads filename (a text file name) and returns the lines of text as a list
  
  Args:
    filename: a string containing the target filepath

  Returns:
    a list of strings with one string per line from the target filename.
  """

  with open(filename,'r') as f:
    return f.readlines()




In [16]:
train_lines = getlines(data_dir + 'train.txt')

train_lines[:20]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

In [17]:
len(train_lines)

210040

In [ ]:
[{'line_number':0,
  'target':'BACKGROUND',
  'text':'Emotional eating is associated with overeating and the development of obesity .\n',
  'total_lines':11},
 ]

In [19]:
def preprocess_text_with_line_numbers(filename):
  """
  Returns a list of dictionaries of abstract line data.
  Takes in filename, reads it contents and sorts through each line,
  extracting thngs like the target label, the text of sentence, 
  how many sentences there are in the current abstract and 
  what sentence number the target line is.
  """

  input_lines = getlines(filename)
  abstract_lines = ''
  abstract_samples = []

  for line in input_lines:
    if line.startswith('###'):
      abstract_id = line
      abstract_lines = ''
    elif line.isspace():
      abstract_line_split = abstract_lines.splitlines()
      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {}
        target_text_split = abstract_line.split('\t')
        line_data['target'] = target_text_split[0]
        line_data['text'] = target_text_split[1].lower()
        line_data['line_number'] = abstract_line_number
        line_data['total_lines'] = len(abstract_line_split) -1
        abstract_samples.append(line_data)

    else:
      abstract_lines += line

  return abstract_samples

In [24]:
%%time

train_samples = preprocess_text_with_line_numbers(data_dir + 'train.txt')
test_samples = preprocess_text_with_line_numbers(data_dir + 'test.txt')
validate_samples = preprocess_text_with_line_numbers(data_dir + 'dev.txt')

print(len(train_samples), len(test_samples), len(validate_samples))

180040 30135 30212
CPU times: user 372 ms, sys: 88.5 ms, total: 461 ms
Wall time: 463 ms
